In [2]:
import numpy as np
import  MyToolSet as myfunc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.base import BaseEstimator, TransformerMixin


class CustomImputation(BaseEstimator, TransformerMixin):
    def __init__(self, ignore=[]):
        self.ignore= ignore

    def fit(self, X, y=None):
        myfunc.missing_values_table(X)
        return self

    def transform(self, X):
        return myfunc.impute_all(X,self.ignore)


class CustomOutlierRemoval(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df_original,dfprocessed=myfunc.outlier_detection(X)
        return dfprocessed

class CustomOutlierRemoverbyZscore(BaseEstimator,TransformerMixin):
    def __init__(self,factor=1.5):
        self.factor = factor
        
    def outlier_removal(self,X,y=None):
        X = pd.Series(X).copy()
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (self.factor * iqr)
        upper_bound = q3 + (self.factor * iqr)
        X.loc[((X < lower_bound) | (X > upper_bound))] = np.nan 
        return pd.Series(X)
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        return X.apply(self.outlier_removal)


class CustomScale(BaseEstimator, TransformerMixin):
    def __init__(self, mode='standard'):
        self.mode=mode
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return myfunc.pre_process(X,mode)
    

class CustomBalanceData(BaseEstimator, TransformerMixin):
    def __init__(self, target,choice=1):
        self.choice=choice
        self.target=target
        if target is None: 
            print('\n Without target variable data cannot be balanced. Specify a target variable.\n')
            
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        return myfunc.BalancedSample(X,self.target,self.choice)

class CustomDimensionReduction(BaseEstimator, TransformerMixin):
    def __init__(self, choice='pca',num=3):
        self.choice=choice
        self.num=num
        
           
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        func=myfunc.My_PrincipalComponentAnalysis if self.choice=='pca' else myfunc.My_FactorAnalyzer
        Loadings,df_pca=func(X,self.num)
        return df_pca

class CustomCluster(BaseEstimator, TransformerMixin):
    def __init__(self,num=5):
        
        self.num=num
        
           
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        Cluster_label=myfunc.my_KMeans(X,num)
        return pd.concat([X,pd.DataFrame(Cluster_label,columns=['Cluster'])],axis=1)

